Movie Recommender System

Steps:

1. import packages and data
2. train hybrid model on MovieLens data

    2a. Content-based for cold start

    2b. Collaborative filtering afterwards
    
3. evaluate model
4. deploy model using flask/etc.
    4a. Api calls to collect user events
    4b. Retrain model with new batch every night

User Perspective:

Simple website that requests an account signup. The user will input 10 movies and their ratings. When they watch movies they will update their profile. They will be provided with 10 recommended movies.



In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [2]:
small = 'data/ml-latest-small'
big = 'data/ml-32m'
folder = big

ratings = pd.read_csv(f'{folder}/ratings.csv')


movies = pd.read_csv(f'{folder}/movies.csv')

In [3]:
# Generates a sparse utility matrix
def create_X(df):
    """
    Args:
        df: pandas dataframe containing 3 columns (userId, movieId, rating)
    
    Returns:
        X: sparse matrix
        user_mapper: dict that maps user id's to user indices
        user_inv_mapper: dict that maps user indices to user id's
        movie_mapper: dict that maps movie id's to movie indices
        movie_inv_mapper: dict that maps movie indices to movie id's
    """
    M = df['userId'].nunique()
    N = df['movieId'].nunique()

    user_mapper = dict(zip(np.unique(df["userId"]), list(range(M))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(N))))
    
    user_inv_mapper = dict(zip(list(range(M)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(N)), np.unique(df["movieId"])))
    
    user_index = [user_mapper[i] for i in df['userId']]
    item_index = [movie_mapper[i] for i in df['movieId']]

    X = csr_matrix((df["rating"], (user_index,item_index)), shape=(M,N))
    
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

In [ ]:

import time

start = time.time()
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)
end = time.time()
print(end-start)
movie_titles = dict(zip(movies['movieId'], movies['title']))


11.8068687915802


In [ ]:
# calculate similarities for unique, highly similar pairs
from sklearn.neighbors import NearestNeighbors
import scipy.sparse as sp
import numpy as np

# X is  user×item sparse rating matrix (csr_matrix of shape [n_users, n_items]).
item_features = X.T.tocsr()  # shape: (n_items, n_users)

# n_neighbors = K+1
K = 15
nn = NearestNeighbors(n_neighbors=K+1, metric="cosine", algorithm="brute", n_jobs=-1)
nn.fit(item_features)

# for each item i, find K+1 nearest neighbors (the first is itself)
distances, indices = nn.kneighbors(item_features, return_distance=True)
# distances[i][0] == 0.0  (item i to itself)
# cosine similarity from distance: similarity = 1 - distance

# build sparse “triplet” table directly
rows = []
cols = []
sims = []
n_items = item_features.shape[0]
for i in range(n_items):
    for rank in range(1, K+1):
        j = indices[i][rank]
        sim_ij = 1.0 - distances[i][rank]
        anchor_id   = movie_inv_mapper[i]
        neighbor_id = movie_inv_mapper[j]

        rows.append(anchor_id)
        cols.append(neighbor_id)
        sims.append(sim_ij)



# similarities stored as (anchor_id, neighbor_id, similarity)

In [ ]:
# use data to make recommendation for user

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import time

seed_movies = [(1, 5.0), (2, 3.5), (3, 5.0),(4, 2.5), (5, 4.0),
               (6, 1.5),  (7, 1.0),  (8, 3.0),  (9, 2.5),  (10, 2.5),
    (11, 2.0), (12, 1.5), (13, 5.0), (14, 1.5), (15, 4.0),
    (16, 1.0), (17, 1.0), (18, 1.5), (19, 2.5), (20, 2.5),
    (21, 5.0), (22, 2.5), (23, 2.5), (24, 3.0), (25, 3.0),
    (26, 4.0), (27, 2.0), (28, 4.5), (29, 1.5), (30, 2.5),
    (31, 4.5), (32, 3.5), (33, 2.0), (34, 1.5), (35, 1.0),
    (36, 2.0), (37, 3.0), (38, 1.0), (39, 4.5), (40, 2.0),
    (41, 3.0), (42, 3.5), (43, 3.0), (44, 1.5), (45, 1.5),
    (46, 3.0), (47, 1.5), (48, 3.5), (49, 2.0), (50, 1.0),
    (51, 2.5), (52, 1.5), (53, 3.5), (54, 1.5), (55, 3.0),
    (57, 3.5), (58, 4.5), (59, 2.5), (60, 3.5),
    (61, 1.5), (62, 3.0), (63, 3.5), (64, 1.0), (65, 3.0),
    (66, 4.5), (67, 2.5), (68, 2.0), (69, 5.0), (70, 4.0),
    (71, 2.5), (72, 1.0), (73, 1.0), (74, 4.0), (75, 3.5),
    (76, 3.0), (77, 3.5), (78, 2.0), (79, 4.0), (80, 3.0),
    (81, 2.0), (82, 4.5), (83, 5.0), (85, 2.0),
    (86, 3.5), (87, 1.5), (88, 2.0), (89, 1.0), (90, 3.5),
    (91, 2.5), (92, 4.5), (93, 4.0), (94, 4.5), (95, 2.0),
    (96, 3.0), (97, 2.0), (98, 2.5), (99, 5.0), (100, 5.0),
    (101, 3.0), (102, 4.0), (103, 4.0), (104, 3.5), (105, 2.5),]


minimum_seed_count = 3
# take all movies rated higher than 4 stars
# if none, take top 3 movies
def find_highly_rated_movies(seed_movies):
    sorted_movies = sorted(seed_movies, key=lambda x: x[1], reverse=True)
    output = []
    for i in range(len(sorted_movies)):
        movie = sorted_movies[i]
        if movie[1] < 4.0 and len(output) > minimum_seed_count:
            break
        output.append(movie[0])
    return output


alpha = 10

# count corated users
def co_rating_count(i_index, j_index):
    col_i = X[:, i_index]
    col_j = X[:, j_index]
    common = col_i.multiply(col_j)
    return common.count_nonzero()


# Get top k movies based on weight of movie
# returns [(movie_id, weighted_sim, raw_sim, co_cnt, TESTING: og_movie_id)]
def topk_from_matrix_weighted(movie_id, S, movie_mapper, movie_inv_mapper, k=15):
    i = movie_mapper[movie_id]
    row = S[i]  # raw SVD-cosine similarities to all items

    # find top (k+1) indices by raw cosine
    idxs = np.argpartition(-row, k+1)[: (k+1)]
    idxs = [j for j in idxs if j != i]
    idxs = sorted(idxs, key=lambda j: -row[j])[:k]

    results = []
    for j in idxs:
        raw_sim = row[j]
        co_cnt  = co_rating_count(i, j)
        shrink  = co_cnt / (co_cnt + alpha)
        if co_cnt < 10:
            weighted_sim = raw_sim * shrink
        else:
            weighted_sim = raw_sim
        results.append((movie_inv_mapper[j], weighted_sim, raw_sim, co_cnt, movie_id))
    # sort final list by descending weighted_sim
    results.sort(key=lambda tup: -tup[1])
    return results

# # print(topk_from_matrix_weighted(1))


# # helper function to produce sorted recommendation list
# def insert_rec(all_recs, rec):
#     score = rec[1]
#     low = 0
#     high = len(all_recs)
#     while low < high:
#         mid = (low + high) // 2

#         if score > all_recs[mid][1]:
#             high = mid
#         else:
#             low = mid + 1
#     all_recs.insert(low, rec)


# start2 = time.time()
# highly_rated_movies = find_highly_rated_movies(seed_movies)
# end2 = time.time()
# print(end2-start2)


# # Collect final list of recommendations
# start3 = time.time()
# all_recs = []
# for i in range(len(highly_rated_movies)):
#     recs = topk_from_matrix_weighted(highly_rated_movies[i], S, movie_mapper, movie_inv_mapper)
#     for rec in recs:
#         insert_rec(all_recs, rec)
# end3 = time.time()
# print(end3-start3)

print('End')

End


In [ ]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.decomposition import TruncatedSVD
# import pickle
# import time



# start = time.time()
# X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_X(ratings)
# end = time.time()
# print(end-start)

# start = time.time()
# svd = TruncatedSVD(n_components=20, n_iter=10)
# Q = svd.fit_transform(X.T)
# Q.shape
# end = time.time()
# print(end-start)

# start = time.time()
# S = cosine_similarity(Q) # similarity matrix
# end = time.time()
# print(end-start)

# start = time.time()
# # Save S to database
# with open('database/similarity_matrix.pkl','wb') as f:
#     pickle.dump(S, f)
# end = time.time()
# print(end - start)

In [10]:


# print('S', S)
# movie_titles = dict(zip(movies['movieId'], movies['title']))
# movie_titles_inv = dict(zip(movies['title'], movies['movieId']))



# movie = 673
# title = movie_titles[movie]

# similar_movies = neighbors.get(movie)[:10]
# # movie_titles = [movie_titles[mov] for mov in similar_movies]
# # print(f'Similar movies to {title}:', movie_titles) 

In [11]:
# later: 
# with open('item_neighbors.pkl','rb') as f:
#     neighbors = pickle.load(f)
# then neighbors[movie_id] gives your k similar movies directly